In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as mp
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [20]:
from google.colab import files
upload=files.upload() # LOADING DATA FROM OUR LOCAL MACHINE

Saving uncleandata.csv to uncleandata (1).csv


In [21]:
# READING THE DATASET AND DISPLAYING ITS CONTENTS
train=pd.read_csv('uncleandata (1).csv', header=0)
train.head()

,tweets,tweet_id,date,likes,retweets,sentiment
0,"Happy Halloween🎃\nCheat the usual price, scare...",1.320000e+18,10/31/2020 8:57,13,0,Positive
1,RT @washingtonpost: Pope Francis calls for civ...,1.320000e+18,10/27/2020 15:13,0,17035,Negative
2,RT @TapasCielo: Kitchen open &amp; enjoy amazi...,1.320000e+18,10/27/2020 10:02,0,16,Positive
3,'Cause love's such an old fashioned word &amp;...,1.320000e+18,10/27/2020 10:01,5,1,Positive
4,Now I can’t get the song out of my head 😄 Mmm ...,1.320000e+18,10/27/2020 9:53,28,1,Negative


In [22]:
train.describe()

,tweet_id,likes,retweets
count,2.000000e+02,200.000000,200.000000
mean,1.231300e+18,213.910000,404.795000
std,4.188174e+16,1362.305536,2843.170059
min,1.180000e+18,0.000000,0.000000
25%,1.190000e+18,1.750000,0.000000
50%,1.230000e+18,11.500000,1.000000
75%,1.250000e+18,55.250000,5.000000
max,1.320000e+18,17244.000000,33156.000000


In [23]:
train.drop(["tweet_id", "date",	"likes",	"retweets",	"sentiment"], axis = 1, inplace = True) 
train.head()

,tweets
0,"Happy Halloween🎃\nCheat the usual price, scare..."
1,RT @washingtonpost: Pope Francis calls for civ...
2,RT @TapasCielo: Kitchen open &amp; enjoy amazi...
3,'Cause love's such an old fashioned word &amp;...
4,Now I can’t get the song out of my head 😄 Mmm ...


In [24]:
#adding header
col= ['tweets']
train.columns=col
train.head()

,tweets
0,"Happy Halloween🎃\nCheat the usual price, scare..."
1,RT @washingtonpost: Pope Francis calls for civ...
2,RT @TapasCielo: Kitchen open &amp; enjoy amazi...
3,'Cause love's such an old fashioned word &amp;...
4,Now I can’t get the song out of my head 😄 Mmm ...


In [25]:
train = train.astype(str)

In [26]:
#Number of Words
train['word_count'] = train['tweets'].apply(lambda x: len(str(x).split(" ")))
train[['tweets','word_count']].head()

#Character Count
train['char_count'] = train['tweets'].str.len() ## this also includes spaces
train[['tweets','char_count']].head()

#Average Word Length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['tweets'].apply(lambda x: avg_word(x))
train[['tweets','avg_word']].head()

#Number of stopwords
import nltk
nltk.download('stopwords')# downlaoding the package.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stop = stopwords.words('english')

train['stopwords'] = train['tweets'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['tweets','stopwords']].head()

#Number of special characters
train['hastags'] = train['tweets'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['tweets','hastags']].head()

#Transform tweets to Lower Case
train['tweets'] = train['tweets'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweets'].head()

#Removing Punctuations
train['tweets'] = train['tweets'].str.replace('[^\w\s]','')
train['tweets'].head()

# Removing stopwords
stop = stopwords.words('english')
train['tweets'] = train['tweets'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['tweets'].head()

#Common Word Removal. 
freq = pd.Series(' '.join(train['tweets']).split()).value_counts()[:10]
freq

train['tweets'] = train['tweets'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['tweets'].head()

train['tweets'] = train['tweets'].str.replace("https", " ")
train['tweets'].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    happy halloween cheat usual price scare hell e...
1    washingtonpost pope francis calls civil union ...
2    tapascielo kitchen open enjoy amazing dishes c...
3    cause loves old fashioned word dares care peop...
4    cant get song head mmm num ba de dum bum ba do...
Name: tweets, dtype: object

In [27]:
freq1 = pd.Series(' '.join(train['tweets']).split()).value_counts()
freq1

freq1 = pd.Series(' '.join(train['tweets']).split()).value_counts()[-13186:]
freq1

freq1 = list(freq1.index)
train['tweets'] = train['tweets'].apply(lambda x: " ".join(x for x in x.split() if x not in freq1))
train['tweets'].head()

check= pd.Series(' '.join(train['tweets']).split()).value_counts()[:]
check

#Spelling Correction
from textblob import TextBlob
train['tweets'[:5223]].apply(lambda x: str(TextBlob(x).correct()))
train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


,tweets,word_count,char_count,avg_word,stopwords,hastags
0,,19,140,6.050000,7,0
1,,13,104,7.076923,2,0
2,,20,144,5.500000,3,3
3,,26,148,4.730769,10,0
4,,27,120,3.481481,4,0


In [28]:
train.describe()

,word_count,char_count,avg_word,stopwords,hastags
count,200.000000,200.00000,200.000000,200.000000,200.000000
mean,13.535000,90.18000,6.875573,4.225000,0.200000
std,8.014984,44.56645,3.257875,3.561996,0.609803
min,1.000000,12.00000,3.481481,0.000000,0.000000
25%,6.000000,47.50000,4.952020,1.000000,0.000000
50%,13.000000,92.50000,5.820802,3.500000,0.000000
75%,21.000000,140.00000,7.407143,7.000000,0.000000
max,28.000000,148.00000,23.000000,14.000000,4.000000


In [32]:
train.to_csv('dataset', encoding='utf-8', index=False)